# Data Scientist Position Testing : **1. USER METRICS**
-   *Name:* Humberto Franco Osorio
-   *Date:* February 6th

## 1th Test

### 1. Import libraries

In [50]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

pd.options.plotting.backend = 'plotly'
pd.set_option('display.float_format', '{:,.2f}'.format) # Number format for pandas

### 2.  Load Dataset

In [51]:
data_raw = pd.read_csv('../data/raw/depositos_oinks.csv')
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4345 entries, 0 to 4344
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        4345 non-null   int64  
 1   user_id           4345 non-null   object 
 2   operation_value   4345 non-null   float64
 3   operation_date    4345 non-null   object 
 4   maplocation_name  4345 non-null   object 
 5   user_createddate  4345 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 203.8+ KB


In [52]:
data_raw.describe()

,Unnamed: 0,operation_value
count,"4,345.00","4,345.00"
mean,"2,172.00","34,618.35"
std,"1,254.44","107,142.69"
min,0.00,50.00
25%,"1,086.00","1,650.00"
50%,"2,172.00","5,600.00"
75%,"3,258.00","22,300.00"
max,"4,344.00","2,595,000.00"


### 3. Data Description and Cleaning

The dataframe used corresponds to deposits made between the dates 2021-11-26 and 2022-02-24 (3 months), at the deposit points called 'CC Plaza de las Américas  - Plaza Mariposa' (Bogota - Colombia), 'Universidad de los Andes - ML Piso 5' (Bogota - Colombia) and 'CC Los Molinos - Zona Montaña Nivel 2' (Medellin - Colombia), 3 deposit points of the approximately 68 that are distributed in Colombia, i.e. deposit report for approximately 4.5% of the deposit equipment installed in Colombia

#### Variable identification

List of features and their descriptions in the initial dataset

| Feature name  | Type    | Description and values   | % missing   |
|---:|:-----------|:------|:------|
| Unnamed: 0            |   Numeric     |  Secuencial number                |   0   |
| user_id               |   Nominal     |  Unique identifier of an user     |   0   |
| operation_value       |   Numeric     |  Deposit amount at alcancy coink  |   0   |
| operation_date        |   Numeric     |  Operation date                   |   0   |
| maplocation_name      |   Nominal     |  Ubication name of                |   0   |
| user_createddate      |   Numeric     |  Creation user date               |   0   |


Removing the not necesary information

In [53]:
data_raw.drop(['Unnamed: 0'],axis=1,inplace=True)   # Drop column 'Unnamed: 0' because don't has information
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4345 entries, 0 to 4344
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           4345 non-null   object 
 1   operation_value   4345 non-null   float64
 2   operation_date    4345 non-null   object 
 3   maplocation_name  4345 non-null   object 
 4   user_createddate  4345 non-null   object 
dtypes: float64(1), object(4)
memory usage: 169.9+ KB


Now, we generate a resume for the data set:
-   Register for deposite of 1656 user.
-   Register of 3 locations.
-   The dataset isn't missing values.

In [54]:
# Chekcing the unique values in each feature
print("-----------------------------------------------------------------------------------")
for col in data_raw:
    print(f"{col} = {data_raw[col].unique()}")
    print("Number option",f"{len(data_raw[col].unique())}")
    print("-----------------------------------------------------------------------------------")

-----------------------------------------------------------------------------------
user_id = ['0e52d550-ae23-407a-9b1f-6e5fb1f066ab'
 '975ed41e-d891-4c23-aeba-06363019d8e3'
 'f9043545-dfc2-402f-a8c7-0a61f21cb719' ...
 '63a492d2-75b7-4238-be98-611b0e496e26'
 'f744b931-6232-43f6-bb7b-bd52275c3ddc'
 'c3d37131-76e8-4c4f-8e55-2778ab8817ad']
Number option 1656
-----------------------------------------------------------------------------------
operation_value = [273850.    900.   1800. ...  40950.  29500. 126200.]
Number option 1232
-----------------------------------------------------------------------------------
operation_date = ['2022-01-14 13:07:56' '2022-02-06 13:11:25' '2022-02-23 14:01:56' ...
 '2021-12-06 19:33:24' '2022-01-24 11:03:38' '2022-02-12 11:58:57']
Number option 4344
-----------------------------------------------------------------------------------
maplocation_name = ['CC Plaza de las Américas  - Plaza Mariposa'
 'Universidad de los Andes - ML Piso 5'
 'CC Los Molinos - 

### 4. Exploratory Data Analysis

#### Metrics for user

In [55]:
print(f'- Amount collected in the three equipments: {data_raw["operation_value"].sum()}')

- Amount collected in the three equipments: 150416750.0


In [56]:
# Generatre new dataset for ploting information about user deposits
new_df = pd.merge(  
    data_raw.groupby('user_id', as_index=False).sum('operation_value'),     # information about total deposit is extrac
    data_raw.groupby('user_id', as_index=False)['operation_date'].count()   # information about quantity of deposits is extract
).rename(columns = {'operation_value':'total_operation_value', 'operation_date': 'quantity_operations'}) 
new_df

,user_id,total_operation_value,quantity_operations
0,001762e3-094d-415c-9304-c7c182e0c16a,"44,750.00",3
1,002a5f4d-a93d-4cc5-b089-e16169bc16a2,"259,000.00",1
2,006687d3-8ff3-461e-bdac-cb8e8e4c5a26,"375,600.00",4
3,0072157e-a634-4cb3-bead-b1a2981485ca,"433,450.00",10
4,0082af19-8bf7-46f7-8a21-c34e46ac5e2a,"1,500.00",1
...,...,...,...
1651,ff2e941c-8808-4219-bdd9-33202e11c0b0,50.00,1
1652,ff5aacc8-320e-4cc3-82a8-059316b857c4,"23,000.00",1
1653,ffaace49-8a5c-4030-a791-8815b5314283,"97,800.00",1
1654,ffe7b3ac-7c57-487d-91bc-de505fdfdcab,"2,500.00",2


In [57]:
new_df.describe()

,total_operation_value,quantity_operations
count,"1,656.00","1,656.00"
mean,"90,831.37",2.62
std,"307,650.47",3.74
min,50.00,1.00
25%,"2,137.50",1.00
50%,"12,400.00",1.00
75%,"69,150.00",3.00
max,"7,032,500.00",61.00


- 75% of users deposited 3 times in the three months or less, 25% deposited 3 or more during the same period.
- 50% of the users deposited a total of 12,400 COP or less, the other 50% deposited a total of 12,400 COP or more.
- The user who deposited the most money during the three months deposited a total of 7,032,500 pesos. And the user who deposited the least, deposited a total of 50 COP.
- 61 times was the maximum amount of deposits made by the same user.

**A. User participation metric:** 
-   "Users frequently use the system to deposit their money":

The boxplot shown, helps to understand that strategies should be sought to promote users to use system for depositing their money, given that 75% of registered customers made 3 or less deposits in the course of 3 months.

-   "Users trust the company to take care of their money":

A total of 150 416 750 COP was collected among the users, which shows that the company has the trust of the users to deposit their money, however, it is necessary to extend this trust to other users since 50% of them deposited 12 400 COP or less in the three months of the study.

However, a considerable amount of deposits is achieved, considering that these were collected by only 4.5% of the equipment installed in the Colombian territory.

In [58]:
trace0 = go.Box(y=new_df['quantity_operations'], name='')
trace1 = go.Box(y=new_df['total_operation_value'], name='')

fig = make_subplots(rows=1, cols=2, subplot_titles=("Box plot of quantity of operations","Box plot of total amount deposited in the platform"))
fig.append_trace(trace0, row = 1, col = 1)
fig.append_trace(trace1, row = 1, col = 2)
fig.update_layout(showlegend=False)
fig.show()

Now, the following graphs show the 50 users who deposited the most money and the ones who deposited the most times, which help to find that in the deposits registered in the dataset, a large amount of deposits does not mean a large amount of money deposited and vice versa.

In [61]:
px.bar(new_df.sort_values('quantity_operations', ascending=False).head(50),
              x = 'user_id',
              y='quantity_operations',
              hover_data=['total_operation_value'], color='total_operation_value',
              title='50 users who deposited the most number of times in the given time period',
              labels={'user_id':'user ID', 'quantity_operations': 'Quantity of deposits', 'total_operation_value': 'Total deposit'}
       )

In [62]:
px.bar(new_df.sort_values('total_operation_value', ascending=False).head(50),
              x = 'user_id',
              y='total_operation_value',
              hover_data=['quantity_operations'], color='quantity_operations',
              title='50 users who deposited the most money in the given time period',
              labels={'user_id':'user ID', 'total_operation_value': 'Total deposit', 'quantity_operations': 'Quantity of deposits' }
       )

**B. User location metric:**

The following images show the summary of the number of deposits and total deposited in each of the three points analyzed.

It can be seen that the main approaches to install collection equipment should be commercial centers where a large number of registered deposits were recorded, and a much larger amount of money was collected in each of the commercial centers compared to the deposit point installed at the university.

In [63]:
data_raw.groupby('maplocation_name').describe()

operation_value            \
                                                     count      mean   
maplocation_name                                                       
CC Los Molinos - Zona Montaña Nivel 2             1,894.00 18,266.97   
CC Plaza de las Américas  - Plaza Mariposa        1,976.00 56,802.13   
Universidad de los Andes - ML Piso 5                475.00  7,532.84   

                                                                      \
                                                  std   min      25%   
maplocation_name                                                       
CC Los Molinos - Zona Montaña Nivel 2       39,625.36 50.00 1,050.00   
CC Plaza de las Américas  - Plaza Mariposa 150,928.83 50.00 2,500.00   
Universidad de los Andes - ML Piso 5        12,830.41 50.00 1,300.00   

                                                                             
                                                 50%       75%          max  
maplocation_name                                                             
CC Los Molinos - Zona Montaña Nivel 2       4,000.00 15,137.50   443,850.00  
CC Plaza de las Américas  - Plaza Mariposa 10,000.00 43,825.00 2,595,000.00  
Universidad de los Andes - ML Piso 5        3,300.00  8,250.00   140,050.00

In [64]:
px.histogram(data_raw,x = 'maplocation_name',
       title='Deposits quantity  at each analysis point',
       labels={'maplocation_name':'Location name'}
)

In [65]:
px.histogram(data_raw,
              x = 'maplocation_name', 
              y='operation_value', 
              title='Money deposited at each analysis point',
              labels={'maplocation_name':'Location name', 'operation_value':'Money deposited' })

**C. Deposit metric:**

The objective of having users deposit a high amount of money per deposit is achieved, where in the "CC Plaza de las Américas - Plaza Mariposa" 50% of the deposits made were for an amount of 10 000 COP or less.

The boxplot graph presented below, allows us to visualize anomalous values of deposit, where a deposit with a value of 2 595 000 COP is evidenced, this graph helps to identify users for surveillance for possible anomalous movements.


In [66]:
px.box(data_raw,x = 'maplocation_name', y='operation_value', color='maplocation_name',
       title='Boxplots edad con relación al sexo')

In [67]:
data_raw['operation_date'] = pd.to_datetime(data_raw['operation_date'])
data_raw['operation_hour'] = data_raw['operation_date'].dt.hour
data_raw['operation_date'] = data_raw['operation_date'].dt.date
data_raw

,user_id,operation_value,operation_date,maplocation_name,user_createddate,operation_hour
0,0e52d550-ae23-407a-9b1f-6e5fb1f066ab,"273,850.00",2022-01-14,CC Plaza de las Américas - Plaza Mariposa,2022-01-09 19:23:18.332689,13
1,975ed41e-d891-4c23-aeba-06363019d8e3,900.00,2022-02-06,CC Plaza de las Américas - Plaza Mariposa,2022-01-10 15:19:29.419075,13
2,f9043545-dfc2-402f-a8c7-0a61f21cb719,"1,800.00",2022-02-23,Universidad de los Andes - ML Piso 5,2021-08-25 12:44:48.524941,14
3,979ca8ad-9600-4a1f-81e9-e70c2f55cdc5,"6,500.00",2021-12-16,CC Plaza de las Américas - Plaza Mariposa,2021-07-17 17:11:16.766291,12
4,171db06e-2e4b-4542-a9c9-32028520fda4,"150,000.00",2022-01-25,CC Plaza de las Américas - Plaza Mariposa,2022-01-11 15:59:27.651994,12
...,...,...,...,...,...,...
4340,c3d37131-76e8-4c4f-8e55-2778ab8817ad,850.00,2021-12-26,CC Los Molinos - Zona Montaña Nivel 2,2021-02-26 09:04:03.443362,13
4341,40bc63c7-4ef1-420e-9e99-6c473b5fe5b1,"1,800.00",2022-01-23,CC Los Molinos - Zona Montaña Nivel 2,2022-01-23 14:31:03.394729,14
4342,b99b3b89-8d45-4656-a950-c85f202897b1,"3,500.00",2021-12-06,CC Plaza de las Américas - Plaza Mariposa,2021-05-14 15:08:33.500127,19
4343,443ffd46-0b2c-4383-9462-cf4b0519b6de,"126,200.00",2022-01-24,CC Los Molinos - Zona Montaña Nivel 2,2021-12-19 16:03:35.472917,11


In [69]:
data_raw.groupby('operation_date', as_index=False).sum('operation_value')

,operation_date,operation_value,operation_hour
0,2021-11-26,"313,400.00",346
1,2021-11-27,"2,396,400.00",968
2,2021-11-28,"748,100.00",524
3,2021-11-29,"848,850.00",615
4,2021-11-30,"977,200.00",689
...,...,...,...
86,2022-02-20,"2,049,550.00",604
87,2022-02-21,"947,400.00",695
88,2022-02-22,"3,358,100.00",1003
89,2022-02-23,"1,733,000.00",673


#### Future jobs
The data set provided gives the possibility to perform analytics such as:
-   Activity of long-time users compared to new users.
-   Analysis of deposit dates to identify day of the week, days of the month, hours of the day, etc., in which a total availability of equipment operation must be guaranteed for deṕosito.